In [7]:
import sys
print(sys.executable)

from lulab.utils.paths import repo_root
print("Repo root:", repo_root())

from lulab.utils.topic import get_topic_dir, topic_paths

TOPIC_DIR = get_topic_dir()
PATHS = topic_paths(TOPIC_DIR)

print(TOPIC_DIR)
print(PATHS["figures"])
print(PATHS["scripts"])
print(PATHS["tex"])
print(PATHS["tables"])
print(PATHS["meta"])


/Users/mloktionov/anaconda3/envs/astro-ai/bin/python
Repo root: /Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab
/Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab/topics/TOP_0001_exoplanet_birth_radius
/Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab/topics/TOP_0001_exoplanet_birth_radius/figures
/Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab/topics/TOP_0001_exoplanet_birth_radius/scripts
/Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab/topics/TOP_0001_exoplanet_birth_radius/tex
/Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab/topics/TOP_0001_exoplanet_birth_radius/tables
/Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab/topics/TOP_0001_exoplanet_birth_radius/meta


In [9]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
# from lulab.viz.export import save_figure

# topic paths
NOTEBOOK_DIR = Path.cwd()
TOPIC_DIR = NOTEBOOK_DIR.parent   # notebooks -> TOPIC_DIR
OUT = TOPIC_DIR / "figures"
OUT.mkdir(parents=True, exist_ok=True)

# dummy data
x = np.linspace(0, 10, 200)
y = np.sin(x)

fig, ax = plt.subplots()
ax.plot(x, y)
ax.set_title("Dummy plot (pipeline check)")
ax.set_xlabel("x")
ax.set_ylabel("sin(x)")

# save
outpath = OUT / "FIG_000_pipeline_check.png"
fig.savefig(outpath, dpi=150, bbox_inches="tight")
# save_figure(fig, PATHS["figures"] / "FIG_000_pipeline_check", formats=("png",))
print("Saved:", outpath)
plt.close(fig)

Saved: /Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab/topics/TOP_0001_exoplanet_birth_radius/figures/FIG_000_pipeline_check.png


In [2]:
# Scan ANIM_*.ipynb notebooks and print every save_animation(...) call as it appears in cells.
# Usage: run this in a repo-aware environment (same kernel as your notebooks).

from __future__ import annotations

from pathlib import Path
import re

import nbformat


# -----------------------------
# Configure: point to your topic root (auto-tries common locations)
# -----------------------------
HERE = Path.cwd()

CANDIDATES = [
    HERE / "topics" / "TOP_0001_exoplanet_birth_radius",
    HERE / "attractor-lab" / "topics" / "TOP_0001_exoplanet_birth_radius",
    HERE,
]

TOPIC_ROOT = next((p for p in CANDIDATES if (p / "i18n").exists() or (p / "notebooks").exists()), HERE)

NOTEBOOKS = [
    "ANIM_001_EN.ipynb",
    "ANIM_002_EN.ipynb",
    "ANIM_003_EN.ipynb",
]

# Search notebooks in likely places
SEARCH_DIRS = [
    TOPIC_ROOT,
    TOPIC_ROOT / "notebooks",
    TOPIC_ROOT / "build",
    TOPIC_ROOT / "src",
]

def find_notebook(name: str) -> Path:
    for d in SEARCH_DIRS:
        if not d.exists():
            continue
        hit = d / name
        if hit.exists():
            return hit
    # fallback: deep search (first hit)
    hits = list(TOPIC_ROOT.rglob(name))
    if hits:
        return hits[0]
    raise FileNotFoundError(f"Notebook not found: {name} under {TOPIC_ROOT}")

# -----------------------------
# Extractor: grab "save_animation(...)" calls, including multiline calls
# -----------------------------
CALL_RE = re.compile(r"save_animation\s*\(", re.M)

def extract_calls(src: str) -> list[str]:
    calls = []
    i = 0
    while True:
        m = CALL_RE.search(src, i)
        if not m:
            break
        start = m.start()

        # walk forward to matching closing parenthesis for this call
        j = m.end()
        depth = 1
        in_str = None
        esc = False

        while j < len(src) and depth > 0:
            ch = src[j]

            if in_str:
                if esc:
                    esc = False
                elif ch == "\\":
                    esc = True
                elif ch == in_str:
                    in_str = None
            else:
                if ch in ("'", '"'):
                    # naive but works well enough for code cells
                    in_str = ch
                elif ch == "(":
                    depth += 1
                elif ch == ")":
                    depth -= 1
            j += 1

        calls.append(src[start:j].strip())
        i = j

    return calls

# -----------------------------
# Main
# -----------------------------
for nb_name in NOTEBOOKS:
    nb_path = find_notebook(nb_name)
    nb = nbformat.read(nb_path, as_version=4)

    print("\n" + "=" * 90)
    print(f"{nb_name}  ::  {nb_path}")
    print("=" * 90)

    found_any = False
    for ci, cell in enumerate(nb.cells):
        if cell.cell_type != "code":
            continue
        src = cell.source or ""
        if "save_animation" not in src:
            continue

        calls = extract_calls(src)
        if not calls:
            # fallback: print any lines containing save_animation
            lines = [ln for ln in src.splitlines() if "save_animation" in ln]
            if lines:
                found_any = True
                print(f"\n--- Cell #{ci} (fallback lines) ---")
                for ln in lines:
                    print(ln)
            continue

        found_any = True
        print(f"\n--- Cell #{ci} ---")
        for k, c in enumerate(calls, 1):
            print(f"[{k}] {c}")

    if not found_any:
        print("(no save_animation calls found)")


ANIM_001_EN.ipynb  ::  /Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab/topics/TOP_0001_exoplanet_birth_radius/notebooks/ANIM_001_EN.ipynb

--- Cell #6 ---
[1] save_animation(anim, out_path_base: Path)

--- Cell #26 ---
[1] save_animation(anim_lr, ANIM_DIR / "ANIM_001_HRD_reveal_LR")
[2] save_animation(anim_rand, ANIM_DIR / "ANIM_001_HRD_reveal_RANDOM")

--- Cell #29 ---
[1] save_animation(anim_strict, ANIM_DIR / "ANIM_002_FeH_STRICT")
[2] save_animation(anim_eq, ANIM_DIR / "ANIM_002_FeH_EQUALIZER")

--- Cell #33 ---
[1] save_animation(anim, ANIM_DIR / "ANIM_003_rbirth_Toy_vs_Minchev_LINEAR")

--- Cell #35 ---
[1] save_animation(anim, ANIM_DIR / "ANIM_003_rbirth_Toy_vs_Minchev_EQ")

--- Cell #37 ---
[1] save_animation(anim, ANIM_DIR / "ANIM_007_summary_2x2")

ANIM_002_EN.ipynb  ::  /Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab/topics/TOP_0001_exoplanet_birth_radius/notebooks/ANIM_002_EN.ipynb

--- Cell #3 (fallback lines) ---
from lulab.anim.defaul